# TF-Agents 强化学习库完整教程

本教程系统性地介绍 TF-Agents 库的核心概念和使用方法。

## 目录
1. [环境配置与导入](#1-环境配置与导入)
2. [核心组件介绍](#2-核心组件介绍)
3. [环境 (Environment)](#3-环境-environment)
4. [策略 (Policy)](#4-策略-policy)
5. [智能体 (Agent)](#5-智能体-agent)
6. [经验回放 (Replay Buffer)](#6-经验回放-replay-buffer)
7. [完整训练示例](#7-完整训练示例)

## 1. 环境配置与导入

首先安装必要的依赖并导入所需的库。

In [ ]:
# 安装依赖（如果需要）
# !pip install tensorflow tf-agents gymnasium matplotlib

In [ ]:
import os
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
from IPython.display import clear_output

# 设置日志级别
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

# TF-Agents 核心导入
from tf_agents.environments import suite_gym
from tf_agents.environments import tf_py_environment
from tf_agents.specs import array_spec
from tf_agents.trajectories import time_step as ts
from tf_agents.policies import random_tf_policy
from tf_agents.networks import q_network
from tf_agents.agents.dqn import dqn_agent
from tf_agents.replay_buffers import tf_uniform_replay_buffer
from tf_agents.trajectories import trajectory
from tf_agents.utils import common

print(f"TensorFlow 版本: {tf.__version__}")
print(f"GPU 可用: {tf.config.list_physical_devices('GPU')}")

## 2. 核心组件介绍

TF-Agents 的设计遵循模块化原则，核心组件包括：

### 2.1 组件关系图

```
┌─────────────────────────────────────────────────────────────────┐
│                         Training Loop                            │
│  ┌───────────┐    ┌──────────────┐    ┌───────────────────────┐ │
│  │Environment│◄──►│    Agent     │◄──►│    Replay Buffer      │ │
│  │           │    │              │    │                       │ │
│  │ - reset() │    │ - policy     │    │ - add_batch()         │ │
│  │ - step()  │    │ - train()    │    │ - sample()            │ │
│  └───────────┘    │ - networks   │    └───────────────────────┘ │
│                   └──────────────┘                              │
└─────────────────────────────────────────────────────────────────┘
```

### 2.2 数据流

1. **数据收集**: Environment → Policy → Trajectory → Replay Buffer
2. **训练更新**: Replay Buffer → Sample → Agent.train() → Network Update

## 3. 环境 (Environment)

环境是智能体交互的世界，定义了状态空间、动作空间和奖励函数。

### 3.1 加载 Gym 环境

In [ ]:
# 加载 CartPole 环境
env_name = "CartPole-v1"

# Python 环境（用于调试）
py_env = suite_gym.load(env_name)

# TensorFlow 环境（用于训练，支持批处理）
tf_env = tf_py_environment.TFPyEnvironment(py_env)

print("环境信息:")
print(f"  名称: {env_name}")
print(f"  观测空间: {tf_env.observation_spec()}")
print(f"  动作空间: {tf_env.action_spec()}")

### 3.2 理解 TimeStep

`TimeStep` 是 TF-Agents 中描述环境状态的核心数据结构：

$$\text{TimeStep} = (\text{step\_type}, \text{reward}, \text{discount}, \text{observation})$$

- `step_type`: FIRST (回合开始), MID (中间), LAST (终止)
- `reward`: 即时奖励 $r_t$
- `discount`: 折扣因子 $\gamma$
- `observation`: 观测 $s_t$

In [ ]:
# 重置环境并获取初始 TimeStep
time_step = tf_env.reset()

print("初始 TimeStep:")
print(f"  step_type: {time_step.step_type}")
print(f"  reward: {time_step.reward}")
print(f"  discount: {time_step.discount}")
print(f"  observation: {time_step.observation}")

### 3.3 环境交互演示

In [ ]:
# 手动执行几步交互
print("环境交互演示:")
time_step = tf_env.reset()

for i in range(5):
    # 随机选择动作
    action = tf.constant([np.random.randint(2)])
    
    # 执行动作
    next_time_step = tf_env.step(action)
    
    print(f"\n步骤 {i+1}:")
    print(f"  动作: {action.numpy()[0]}")
    print(f"  奖励: {next_time_step.reward.numpy()[0]}")
    print(f"  观测: {next_time_step.observation.numpy()[0][:2]}...")
    print(f"  终止: {next_time_step.is_last().numpy()[0]}")
    
    time_step = next_time_step

## 4. 策略 (Policy)

策略定义了智能体的行为方式，将观测映射到动作。

### 4.1 数学定义

$$\pi: \mathcal{S} \to \mathcal{P}(\mathcal{A})$$

- **确定性策略**: $a = \pi(s)$
- **随机策略**: $a \sim \pi(\cdot|s)$

### 4.2 随机策略示例

In [ ]:
# 创建随机策略
random_policy = random_tf_policy.RandomTFPolicy(
    tf_env.time_step_spec(),
    tf_env.action_spec()
)

# 使用策略选择动作
time_step = tf_env.reset()
action_step = random_policy.action(time_step)

print(f"策略选择的动作: {action_step.action.numpy()}")

### 4.3 评估策略性能

In [ ]:
def evaluate_policy(environment, policy, num_episodes=10):
    """
    评估策略的平均回合回报
    
    数学公式:
    平均回报 = (1/N) * Σ G_i
    其中 G_i = Σ r_t 是第 i 个回合的总奖励
    """
    total_return = 0.0
    
    for _ in range(num_episodes):
        time_step = environment.reset()
        episode_return = 0.0
        
        while not time_step.is_last():
            action_step = policy.action(time_step)
            time_step = environment.step(action_step.action)
            episode_return += time_step.reward.numpy()[0]
        
        total_return += episode_return
    
    return total_return / num_episodes

# 评估随机策略
avg_return = evaluate_policy(tf_env, random_policy, num_episodes=10)
print(f"随机策略平均回报: {avg_return:.2f}")

## 5. 智能体 (Agent)

智能体封装了学习算法，包括网络、优化器和训练逻辑。

### 5.1 DQN 算法简介

DQN 使用神经网络逼近 Q 函数：

$$Q(s, a; \theta) \approx \mathbb{E}\left[\sum_{t=0}^{\infty} \gamma^t r_t | s_0=s, a_0=a\right]$$

训练目标（最小化 TD 误差）：

$$\mathcal{L}(\theta) = \mathbb{E}\left[(r + \gamma \max_{a'} Q(s', a'; \theta^-) - Q(s, a; \theta))^2\right]$$

### 5.2 构建 Q 网络

In [ ]:
# 构建 Q 网络
fc_layer_params = (100, 50)  # 两层全连接，分别 100 和 50 个神经元

q_net = q_network.QNetwork(
    tf_env.observation_spec(),
    tf_env.action_spec(),
    fc_layer_params=fc_layer_params
)

print("Q 网络结构:")
print(f"  输入: {tf_env.observation_spec()}")
print(f"  输出: {tf_env.action_spec()}")
print(f"  隐藏层: {fc_layer_params}")

### 5.3 创建 DQN 智能体

In [ ]:
# 超参数配置
learning_rate = 1e-3
gamma = 0.99
epsilon_greedy = 0.1
target_update_period = 200

# 创建优化器
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

# 训练步数计数器
train_step_counter = tf.Variable(0, dtype=tf.int64)

# 创建 DQN 智能体
agent = dqn_agent.DqnAgent(
    tf_env.time_step_spec(),
    tf_env.action_spec(),
    q_network=q_net,
    optimizer=optimizer,
    td_errors_loss_fn=common.element_wise_squared_loss,
    train_step_counter=train_step_counter,
    gamma=gamma,
    epsilon_greedy=epsilon_greedy,
    target_update_period=target_update_period
)

agent.initialize()

print("DQN 智能体配置:")
print(f"  学习率: {learning_rate}")
print(f"  折扣因子 γ: {gamma}")
print(f"  探索率 ε: {epsilon_greedy}")
print(f"  目标网络更新周期: {target_update_period}")

## 6. 经验回放 (Replay Buffer)

经验回放存储历史交互数据，通过随机采样打破样本相关性。

### 6.1 数学原理

缓冲区存储经验元组：
$$\mathcal{D} = \{(s_i, a_i, r_i, s'_i, \text{done}_i)\}_{i=1}^{N}$$

均匀采样：
$$P(i) = \frac{1}{|\mathcal{D}|}$$

### 6.2 创建回放缓冲区

In [ ]:
# 创建回放缓冲区
replay_buffer_capacity = 100000

replay_buffer = tf_uniform_replay_buffer.TFUniformReplayBuffer(
    data_spec=agent.collect_data_spec,
    batch_size=tf_env.batch_size,
    max_length=replay_buffer_capacity
)

print(f"回放缓冲区容量: {replay_buffer_capacity}")
print(f"数据规范: {agent.collect_data_spec}")

### 6.3 数据收集函数

In [ ]:
def collect_step(environment, policy, buffer):
    """
    收集单步交互数据并存入缓冲区
    
    流程:
    1. 获取当前状态 s_t
    2. 策略选择动作 a_t
    3. 环境执行动作，返回 (r_t, s_{t+1})
    4. 构建 Trajectory 并存入缓冲区
    """
    time_step = environment.current_time_step()
    action_step = policy.action(time_step)
    next_time_step = environment.step(action_step.action)
    
    # 构建轨迹
    traj = trajectory.from_transition(time_step, action_step, next_time_step)
    
    # 存入缓冲区
    buffer.add_batch(traj)

def collect_data(environment, policy, buffer, num_steps):
    """收集多步数据"""
    for _ in range(num_steps):
        collect_step(environment, policy, buffer)

### 6.4 预填充缓冲区

In [ ]:
# 使用随机策略预填充缓冲区
initial_collect_steps = 1000

tf_env.reset()
collect_data(tf_env, random_policy, replay_buffer, initial_collect_steps)

print(f"缓冲区当前大小: {replay_buffer.num_frames().numpy()}")

## 7. 完整训练示例

将所有组件组合，实现完整的 DQN 训练循环。

### 7.1 训练配置

In [ ]:
# 训练超参数
num_iterations = 10000  # 总训练迭代数
collect_steps_per_iteration = 1  # 每次迭代收集的步数
batch_size = 64  # 训练批大小
log_interval = 200  # 日志打印间隔
eval_interval = 1000  # 评估间隔
num_eval_episodes = 10  # 评估回合数

print("训练配置:")
print(f"  总迭代数: {num_iterations}")
print(f"  批大小: {batch_size}")
print(f"  评估间隔: {eval_interval}")

### 7.2 训练循环

In [ ]:
# 创建评估环境
eval_py_env = suite_gym.load(env_name)
eval_tf_env = tf_py_environment.TFPyEnvironment(eval_py_env)

# 准备数据集
dataset = replay_buffer.as_dataset(
    num_parallel_calls=3,
    sample_batch_size=batch_size,
    num_steps=2  # 连续两步用于计算 TD 目标
).prefetch(3)

iterator = iter(dataset)

# 编译训练函数（加速）
agent.train = common.function(agent.train)

# 记录训练历史
returns = []
losses = []

# 初始评估
avg_return = evaluate_policy(eval_tf_env, agent.policy, num_eval_episodes)
returns.append(avg_return)
print(f"初始平均回报: {avg_return:.2f}")

In [ ]:
# 训练循环
tf_env.reset()

for iteration in range(1, num_iterations + 1):
    # 1. 收集数据
    for _ in range(collect_steps_per_iteration):
        collect_step(tf_env, agent.collect_policy, replay_buffer)
    
    # 2. 采样并训练
    experience, _ = next(iterator)
    train_loss = agent.train(experience).loss
    losses.append(train_loss.numpy())
    
    step = agent.train_step_counter.numpy()
    
    # 3. 日志
    if step % log_interval == 0:
        print(f"Step {step}: Loss = {train_loss:.4f}")
    
    # 4. 评估
    if step % eval_interval == 0:
        avg_return = evaluate_policy(eval_tf_env, agent.policy, num_eval_episodes)
        returns.append(avg_return)
        print(f"Step {step}: Avg Return = {avg_return:.2f}")

print("\n训练完成!")
print(f"最终平均回报: {returns[-1]:.2f}")

### 7.3 可视化训练曲线

In [ ]:
# 绘制训练曲线
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# 回报曲线
ax1 = axes[0]
iterations_eval = [0] + list(range(eval_interval, num_iterations + 1, eval_interval))
ax1.plot(iterations_eval, returns, 'b-o', linewidth=2, markersize=4)
ax1.set_xlabel('Training Iteration', fontsize=12)
ax1.set_ylabel('Average Return', fontsize=12)
ax1.set_title('DQN Training Progress', fontsize=14)
ax1.grid(True, alpha=0.3)

# 损失曲线（平滑）
ax2 = axes[1]
window = 100
smoothed_losses = np.convolve(losses, np.ones(window)/window, mode='valid')
ax2.plot(smoothed_losses, 'r-', linewidth=1, alpha=0.8)
ax2.set_xlabel('Training Step', fontsize=12)
ax2.set_ylabel('Loss (smoothed)', fontsize=12)
ax2.set_title('Training Loss', fontsize=14)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

### 7.4 测试训练好的智能体

In [ ]:
# 测试训练好的策略
print("测试训练好的智能体:")
print("=" * 40)

for episode in range(3):
    time_step = eval_tf_env.reset()
    episode_return = 0.0
    steps = 0
    
    while not time_step.is_last():
        action_step = agent.policy.action(time_step)
        time_step = eval_tf_env.step(action_step.action)
        episode_return += time_step.reward.numpy()[0]
        steps += 1
    
    print(f"回合 {episode + 1}: 步数 = {steps}, 总奖励 = {episode_return:.0f}")

print("=" * 40)

## 总结

本教程介绍了 TF-Agents 的核心组件：

1. **Environment**: 定义状态空间、动作空间和奖励函数
2. **Policy**: 将观测映射到动作
3. **Agent**: 封装学习算法和网络
4. **Replay Buffer**: 存储和采样经验数据

### 关键数学公式回顾

**Q-Learning 更新**:
$$Q(s,a) \leftarrow Q(s,a) + \alpha [r + \gamma \max_{a'} Q(s',a') - Q(s,a)]$$

**DQN 损失函数**:
$$\mathcal{L}(\theta) = \mathbb{E}[(r + \gamma \max_{a'} Q_{\theta^-}(s',a') - Q_\theta(s,a))^2]$$

### 下一步学习

- 查看 `02_dqn_cartpole.py` 了解完整的 DQN 实现
- 学习 `03_sac_continuous_control.py` 了解连续控制
- 探索 `04_ppo_agent.py` 学习策略梯度方法